In [1]:
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [2]:
def naive(p, t):
    occurrences = []
    counter = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        forward = False
        rev = False
        for j in range(len(p)):  # loop over characters
            counter += 1
            if forward == False and rev == False:
                if t[i+j] != p[j] and t[i+j] != reverseComplement(p)[j]:  # compare characters
                    match = False
                    break
                if p[j] == reverseComplement(p)[j]:
                    continue;
                if p[j] != reverseComplement(p)[j] and t[i+j] == p[j]:
                    forward = True
                    continue;
                if p[j] != reverseComplement(p)[j] and t[i+j] == reverseComplement(p)[j]:
                    rev = True
                    continue;
            elif rev == True:
                counter += 1
                if t[i+j] != reverseComplement(p)[j]:
                    match = False
                    break;
            else:
                if t[i+j] != p[j]:
                    match = False
                    break;
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences, counter

In [3]:
def readGenome(filename):
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [49]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences

In [5]:
# naive exact matching, allowing 2 mismatches
def naive_2mm(pattern, ref):
    occurrences = []
    for i in range(len(ref) - len(pattern) + 1):  # loop over alignments
        match = True
        mismatch_count = 0 # mismatch counter
        for j in range(len(pattern)):  # loop over characters
            if ref[i+j] != pattern[j]:  # compare characters
                if not mismatch_count >= 2: # if mismatch count is less than 2, continue
                    mismatch_count += 1
                    continue;
                else:
                    match = False # if mismatch count more than 2, break from loop
                    break;
        if match:
            occurrences.append(i)  # all chars matched; record
    return occurrences

In [6]:
def boyer_moore(p, p_bm, t):
    """ Do Boyer-Moore matching. p=pattern, t=text,
        p_bm=BoyerMoore object for p """
    i = 0
    occurrences = []
    while i < len(t) - len(p) + 1:
        shift = 1
        mismatched = False
        for j in range(len(p)-1, -1, -1):
            if p[j] != t[i+j]:
                skip_bc = p_bm.bad_character_rule(j, t[i+j])
                skip_gs = p_bm.good_suffix_rule(j)
                shift = max(shift, skip_bc, skip_gs)
                mismatched = True
                break
        if not mismatched:
            occurrences.append(i)
            skip_gs = p_bm.match_skip()
            shift = max(shift, skip_gs)
        i += shift
    return occurrences

In [7]:
def naive_occurences_only(p, t):
    counter = 0
    for i in range(len(t) - len(p) + 1):  # loop over alignments
        match = True
        if t[i] == p[0]:
            counter += 1
        for j in range(len(p)):  # loop over characters
            if t[i+j] != p[j]:  # compare characters
                match = False
                break
    return counter

In [8]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [9]:
def editDistance(x, y):
    # Create distance matrix
    D = []
    for i in range(len(x)+1):
        D.append([0]*(len(y)+1))
    # Initialize first row and column of matrix
    for i in range(len(x)+1):
        D[i][0] = i
    #for i in range(len(y)+1):
        #D[0][i] = i
    # Fill in the rest of the matrix
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            distHor = D[i][j-1] + 1
            distVer = D[i-1][j] + 1
            if x[i-1] == y[j-1]:
                distDiag = D[i-1][j-1]
            else:
                distDiag = D[i-1][j-1] + 1
            D[i][j] = min(distHor, distVer, distDiag)
    # Edit distance is the value in the bottom right corner of the matrix
    final = D[-1][0]
    for loop in range(len(y)+1):
        if final > D[-1][loop]:
            final = D[-1][loop]
        else:
            pass
    return final

In [10]:
#chromo = readGenome("chr1.GRCh38.excerpt.fasta")

In [11]:
#editDistance("GATTTACCAGATTGAG", chromo)

In [12]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [40]:
import itertools

def scs(ss):
    """ Returns shortest common superstring of given
        strings, which must be the same length """
    count = []
    shortest_sup = None
    for ssperm in itertools.permutations(ss):
        sup = ssperm[0]  # superstring starts as first string
        for i in range(len(ss)-1):
            # overlap adjacent strings A and B in the permutation
            olen = overlap(ssperm[i], ssperm[i+1], min_length=1)
            # add non-overlapping portion of B to superstring
            sup += ssperm[i+1][olen:]
        if shortest_sup is None or len(sup) <= len(shortest_sup):
            if shortest_sup is not None and len(sup) < len(shortest_sup):
                count.clear()
                count.append(sup)
            elif shortest_sup is not None and len(sup) == len(shortest_sup):
                count.append(sup)
            shortest_sup = sup  # found shorter superstring
    return shortest_sup, count  # return shortest

In [44]:
seq = readFastq("ads1_week4_reads.fq")

In [46]:
def pick_maximal_overlap(reads, k):
    reada, readb = None, None
    best_olen = 0
    for a, b in itertools.permutations(reads, 2):
        olen = overlap(a, b, min_length = k)
        if olen > best_olen:
            reada, readb = a,b 
            best_olen = olen
    return reada, readb, best_olen

def greedy_scs(reads, k):
    """ Greedy shortest-common-superstring merge.
        Repeat until no edges (overlaps of length >= k)
        remain. """
    read_a, read_b, olen = pick_maximal_overlap(reads, k)
    while olen > 0:
        reads.remove(read_a)
        reads.remove(read_b)
        reads.append(read_a + read_b[olen:])
        read_a, read_b, olen = pick_maximal_overlap(reads, k)
    return ''.join(reads)

In [119]:
def overlap_all_pairs(reads, k):
    d = {}
    set2 = set()
    l = []
    for i in range(len(reads)):
        set1 = set()
        for j in range(len(reads[i])):
            if j+k < len(reads[i]):
                set1.add(reads[i][j:j+k])
        d[reads[i]] = set1
    for a, b in itertools.permutations(reads, 2):
        if a != b:
            for i in range(len(d[a])):
                for j in range(len(d[b])):
                    if i == j:
                        overlap(a, b, 3)
                        l.append((a,b))
                        break
    yesh = set(l)
    length = int(len(yesh)/2)
    return set(itertools.islice(yesh, length))

In [50]:
seq1 = readFastq("ERR266411_1.for_asm.fastq")

In [115]:
reads = ['CGTACG', 'TACGTA', 'GTACGT', 'ACGTAC', 'GTACGA', 'TACGAT']

In [120]:
overlap_all_pairs(reads, 4)

{('ACGTAC', 'GTACGA'),
 ('CGTACG', 'ACGTAC'),
 ('CGTACG', 'TACGTA'),
 ('GTACGT', 'GTACGA'),
 ('TACGTA', 'GTACGA'),
 ('TACGTA', 'GTACGT'),
 ('TACGTA', 'TACGAT')}